# Dings Trader — Full Pipeline (All-in-One)
Ein einziges Notebook für den kompletten Ablauf.

**Empfohlene Runtime-Strategie:**
- Schritte 0–3 auf **CPU/T4** (spart Credits)
- dann Runtime auf **A100** wechseln
- danach Schritte 4–7 ausführen

Alle Schritte schreiben Logs nach `logs/colab/full_pipeline.log` und drucken `OK:` / `ERROR:`.


## 0) Setup (Drive + Symlinks + Dependencies)


In [1]:
step = "00_drive_mount"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"OK: {step}")
except Exception as e:
    print(f"ERROR: {step} :: {e}")
    raise


Mounted at /content/drive
OK: 00_drive_mount


In [2]:
%%bash
    set -euo pipefail
    STEP="00_setup_symlinks_and_deps"
    ROOT="${ROOT:-/content/Dings_Trader/TraderHimSelf}"
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    STORE="${STORE:-/content/drive/MyDrive/dings-trader-store/TraderHimSelf}"
mkdir -p "$STORE"/{data_raw,data_processed,models,logs,runs,checkpoints}

for d in data_raw data_processed models logs runs checkpoints; do
  [ -L "$d" ] && rm "$d" || true
  if [ -e "$d" ] && [ ! -L "$d" ]; then
    if [ -n "$(ls -A "$d" 2>/dev/null || true)" ]; then
      mv "$d" "${d}.backup.$(date +%s)"
    else
      rm -rf "$d"
    fi
  fi
  ln -s "$STORE/$d" "$d"
done

python -m pip install -r requirements.txt 2>&1 | tee -a "$LOG"
readlink -f data_raw | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"


bash: line 4: cd: /content/Dings_Trader/TraderHimSelf: No such file or directory


CalledProcessError: Command 'b'    set -euo pipefail\n    STEP="00_setup_symlinks_and_deps"\n    ROOT="${ROOT:-/content/Dings_Trader/TraderHimSelf}"\n    cd "$ROOT"\n    mkdir -p logs/colab\n    LOG="logs/colab/full_pipeline.log"\n    trap \'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc\' ERR\n\n    echo "START: ${STEP}" | tee -a "$LOG"\n    STORE="${STORE:-/content/drive/MyDrive/dings-trader-store/TraderHimSelf}"\nmkdir -p "$STORE"/{data_raw,data_processed,models,logs,runs,checkpoints}\n\nfor d in data_raw data_processed models logs runs checkpoints; do\n  [ -L "$d" ] && rm "$d" || true\n  if [ -e "$d" ] && [ ! -L "$d" ]; then\n    if [ -n "$(ls -A "$d" 2>/dev/null || true)" ]; then\n      mv "$d" "${d}.backup.$(date +%s)"\n    else\n      rm -rf "$d"\n    fi\n  fi\n  ln -s "$STORE/$d" "$d"\ndone\n\npython -m pip install -r requirements.txt 2>&1 | tee -a "$LOG"\nreadlink -f data_raw | tee -a "$LOG"\n    echo "OK: ${STEP}" | tee -a "$LOG"\n'' returned non-ze

## 1) Download Data (CPU/T4)


In [ ]:
%%bash
    set -euo pipefail
    STEP="01_download_data"
    ROOT="${ROOT:-/content/Dings_Trader/TraderHimSelf}"
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    START_DATE="${START_DATE:-2019-01-01}"
END_DATE="${END_DATE:-$(date +%F)}"
SYMBOL="${SYMBOL:-BTCUSDT}"

python download_binance_data.py   --data-dir data_raw   --start-date "$START_DATE"   --end-date "$END_DATE"   --symbol "$SYMBOL" 2>&1 | tee -a "$LOG"

ls -lah data_raw | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"


## 2) Build Dataset (CPU/T4)


In [5]:
%%bash
set -euo pipefail
STEP="02_build_dataset"
ROOT="${ROOT:-/content/Dings_Trader/TraderHimSelf}"
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python build_dataset.py 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"


START: 02_build_dataset
OK: 02_build_dataset


## 3) Feature Engine (CPU/T4)


In [6]:
%%bash
set -euo pipefail
STEP="03_feature_engine"
ROOT="${ROOT:-/content/Dings_Trader/TraderHimSelf}"
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python feature_engine.py build 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"


START: 03_feature_engine
Wrote data_processed/features.parquet and data_processed/scaler.pkl
OK: 03_feature_engine


## 3.5) Runtime Switch auf A100
Jetzt Runtime auf **A100** wechseln (Runtime → Change runtime type).
Nach Reconnect die **nächsten zwei Zellen erneut** ausführen (Drive mount + setup), dann mit Schritt 4 weiter.


In [1]:
step = "35_drive_remount_after_switch"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"OK: {step}")
except Exception as e:
    print(f"ERROR: {step} :: {e}")
    raise


Mounted at /content/drive
OK: 35_drive_remount_after_switch


In [2]:
%%bash
set -euo pipefail

REPO_DIR="/content/Dings_Trader"
REPO_URL="https://github.com/MidnightCoffeex/Dings_Trader.git"

cd /content
if [ -d "$REPO_DIR/.git" ]; then
  echo "Repo existiert -> update auf neuesten Stand"
  cd "$REPO_DIR"
  git fetch origin main
  git reset --hard origin/main
else
  echo "Repo fehlt -> clone"
  git clone "$REPO_URL" Dings_Trader
  cd "$REPO_DIR"
fi

cd /content/Dings_Trader/TraderHimSelf
ls -lah requirements.txt
git log --oneline -n 1
echo "OK: repo ready (latest main)"

Repo fehlt -> clone
-rw-r--r-- 1 root root 361 Feb 11 19:35 requirements.txt
a5517bd Mit Colab erstellt
OK: repo ready (latest main)


Cloning into 'Dings_Trader'...


In [3]:
#relink + deps

%%bash
set -euo pipefail
ROOT="/content/Dings_Trader/TraderHimSelf"
STORE="/content/drive/MyDrive/dings-trader-store/TraderHimSelf"

cd "$ROOT"
mkdir -p "$STORE"/{data_raw,data_processed,models,logs,runs,checkpoints}

for d in data_raw data_processed models logs runs checkpoints; do
  [ -L "$d" ] && rm "$d" || true
  [ -e "$d" ] && [ ! -L "$d" ] && rm -rf "$d"
  ln -s "$STORE/$d" "$d"
done

python -m pip install -r requirements.txt
echo "OK: relink after switch"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.0/144.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.9 MB/s eta 0:00:00
OK: relink after switch


# 04_train_patchtst
Train forecast model.

In [14]:
%%bash
set -euo pipefail

STEP="04_train_patchtst"

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi

cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/${STEP}.log"

trap 'rc=$?; echo "FAIL: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: build_train30_features" | tee -a "$LOG"
python feature_engine_train30.py build 2>&1 | tee -a "$LOG"
echo "OK: build_train30_features" | tee -a "$LOG"

echo "START: ${STEP}" | tee -a "$LOG"
python forecast/train_patchtst.py train \
  --profile high-util \
  --batch-size 512 \
  --num-workers 16 \
  --prefetch-factor 4 \
  --pin-memory true \
  --persistent-workers true \
  --amp true \
  --compile true \
  --log-every-batches 1 2>&1 | tee -a "$LOG"

echo "OK: ${STEP}" | tee -a "$LOG"

START: build_train30_features
[feature_engine_train30] wrote: data_processed/train30/features.parquet  shape=(225238, 30)
[feature_engine_train30] wrote: data_processed/train30/scaler.pkl
OK: build_train30_features
START: 04_train_patchtst
2026-02-11 16:44:13,182 - Forecast-PatchTST - INFO - Configuration: {'batch_size': 512, 'num_workers': 16, 'pin_memory': True, 'prefetch_factor': 4, 'persistent_workers': True, 'amp': True, 'compile': True}
2026-02-11 16:44:13,318 - Forecast-PatchTST - INFO - Using device: cuda
2026-02-11 16:44:13,318 - Forecast-PatchTST - INFO - AMP enabled.
2026-02-11 16:44:13,567 - Forecast-PatchTST - WARNING - Dropping 672 rows with NaN/Inf in features or close before training.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that

## 5) Precompute Forecast Features (A100)


In [21]:
%%bash
set -euo pipefail
cd /content/Dings_Trader
git fetch origin main
git reset --hard origin/main
echo "OK: repo updated"

HEAD is now at c741fdf fix(precompute): support loading torch.compile-prefixed PatchTST checkpoints
OK: repo updated


From https://github.com/MidnightCoffeex/Dings_Trader
 * branch            main       -> FETCH_HEAD
   ce54736..c741fdf  main       -> origin/main


In [15]:
%%bash
set -euo pipefail
STEP="05_precompute_forecast"

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden."
  exit 1
fi

cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/${STEP}.log"
trap 'rc=$?; echo "FAIL: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python forecast/train_patchtst.py precompute \
  --batch-size 1024 \
  --num-workers 12 \
  --amp true \
  --pin-memory true 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"

START: 05_precompute_forecast
2026-02-11 18:25:35,048 - Forecast-PatchTST - INFO - Precompute Configuration: {'batch_size': 1024, 'num_workers': 12, 'pin_memory': True, 'prefetch_factor': None, 'persistent_workers': False, 'amp': True, 'compile': False}
2026-02-11 18:25:35,185 - Forecast-PatchTST - INFO - Starting Precompute...
2026-02-11 18:25:35,426 - Forecast-PatchTST - WARNING - Dropping 672 rows with NaN/Inf in features or close before training.
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
2026-02-11 18:26:15,023 - Forecast-PatchTST - INFO - Saved forecast features to /content/Dings_Trader/TraderHimSelf/data_processed/train30/forecast_features.parquet. Shape: (224566, 35)
OK: 05_precompute_forecast


# 06_train_ppo
Train PPO policy.

In [26]:
%%bash
set -euo pipefail
cd /content/Dings_Trader
git fetch origin main
git reset --hard origin/main
echo "OK: repo updated"

HEAD is now at 363c375 fix(ppo): drop non-finite rows before training and fail fast on invalid observations
OK: repo updated


From https://github.com/MidnightCoffeex/Dings_Trader
 * branch            main       -> FETCH_HEAD
   d23b703..363c375  main       -> origin/main


###OPTIONAL CPU Training

In [5]:
%%bash
set -euo pipefail
STEP="06_train_ppo_cpu_live"

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden."
  exit 1
fi

cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/${STEP}.log"
trap 'rc=$?; echo "FAIL: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python policy/train_ppo.py \
  --device cpu \
  --profile high-util \
  --n-envs 8 \
  --n-steps 8192 \
  --batch-size 8192 \
  --vec-env subproc \
  --allow-dummy-forecast 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"

START: 06_train_ppo_cpu_live
2026-02-11 19:39:19.219985: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-11 19:39:19.225046: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-11 19:39:19.238928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770838759.263374    1956 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770838759.270870    1956 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770838759.290576    1956 computation_placer.cc:177] computation placer already registered. Please c

###OPTIONAL GPU Training

Für Training im Terminal sehen:
"tail -f /content/Dings_Trader/TraderHimSelf/logs/colab/06_train_ppo_gpu_optional.log"



In [20]:
%%bash
set -euo pipefail
STEP="06_train_ppo_gpu_optional"

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden."
  exit 1
fi

cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/${STEP}.log"
trap 'rc=$?; echo "FAIL: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python policy/train_ppo.py \
  --device cuda \
  --profile high-util \
  --n-envs 8 \
  --n-steps 8192 \
  --batch-size 8192 \
  --vec-env subproc \
  --allow-dummy-forecast 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"

Process is terminated.


## 7) Final Report (copy/paste to chat)


In [29]:
%%bash
set -euo pipefail
STEP="07_final_report"

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden."
  exit 1
fi

cd "$ROOT"
mkdir -p logs/colab reports
LOG="logs/colab/${STEP}.log"
OUT="reports/final_report_$(date +%F_%H-%M-%S).md"
trap 'rc=$?; echo "FAIL: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
{
  echo "# Final Report"
  echo
  echo "## Zeit"
  date
  echo
  echo "## Letzte Commits"
  git log --oneline -n 8 || true
  echo
  echo "## Artefakte"
  ls -lah models || true
  ls -lah data_processed/train30 || true
  echo
  echo "## Colab Logs"
  ls -lah logs/colab || true
} | tee "$OUT" | tee -a "$LOG"

echo "OK: ${STEP} -> $OUT" | tee -a "$LOG"

START: 07_final_report
REPORT_START
DINGS_TRADER_REPORT v1
generated_at=2026-02-11T01:18:25.075405+00:00

[FILES]
- raw_15m: OK (7.5MB) :: /content/Dings_Trader/TraderHimSelf/data_raw/btcusdt_15m.parquet
- raw_3m: OK (25.5MB) :: /content/Dings_Trader/TraderHimSelf/data_raw/btcusdt_3m.parquet
- raw_funding: OK (81.1KB) :: /content/Dings_Trader/TraderHimSelf/data_raw/btcusdt_funding.parquet
- aligned_15m: OK (10.0MB) :: /content/Dings_Trader/TraderHimSelf/data_processed/aligned_15m.parquet
- aligned_3m: OK (45.0MB) :: /content/Dings_Trader/TraderHimSelf/data_processed/aligned_3m.parquet
- features: OK (45.9MB) :: /content/Dings_Trader/TraderHimSelf/data_processed/features.parquet
- forecast_features: OK (10.4MB) :: /content/Dings_Trader/TraderHimSelf/data_processed/forecast_features.parquet
- scaler: OK (1.3KB) :: /content/Dings_Trader/TraderHimSelf/data_processed/scaler.pkl
- forecast_model: OK (444.5MB) :: /content/Dings_Trader/TraderHimSelf/models/forecast_model.pt

[DATA_SUMMARY]
- a

In [31]:
%%bash
set -euo pipefail

if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden."
  exit 1
fi

echo "FINAL_EVAL_START"
echo "root=$ROOT"

python - "$ROOT" <<'PY'
import os, re, glob, statistics, sys
root = sys.argv[1]

logs = sorted(
    glob.glob(os.path.join(root, "logs", "colab", "06_train_ppo*.log")),
    key=os.path.getmtime,
    reverse=True
)
if not logs:
    print("status=FAIL")
    print("reason=no_ppo_log_found")
    raise SystemExit(0)

log_file = logs[0]
txt = open(log_file, "r", encoding="utf-8", errors="ignore").read()

def vals(name):
    pat = rf"\|\s*{re.escape(name)}\s*\|\s*([\-0-9.eE]+)\s*\|"
    return [float(x) for x in re.findall(pat, txt)]

def last(v): return v[-1] if v else None
def med10(v):
    if not v: return None
    vv = v[-10:] if len(v) >= 10 else v
    return statistics.median(vv)

fps = vals("fps")
ts  = vals("total_timesteps")
ev  = vals("explained_variance")
kl  = vals("approx_kl")
cf  = vals("clip_fraction")

model_path = os.path.join(root, "models", "ppo_policy_final.zip")
ckpts = glob.glob(os.path.join(root, "checkpoints", "ppo", "*.zip"))

print("status=OK")
print(f"log_file={os.path.basename(log_file)}")
print(f"timesteps_last={int(last(ts)) if last(ts) is not None else 'NA'}")
print(f"fps_last={last(fps) if last(fps) is not None else 'NA'}")
print(f"fps_median_last10={med10(fps) if med10(fps) is not None else 'NA'}")
print(f"explained_variance_last={last(ev) if last(ev) is not None else 'NA'}")
print(f"approx_kl_last={last(kl) if last(kl) is not None else 'NA'}")
print(f"clip_fraction_last={last(cf) if last(cf) is not None else 'NA'}")
print(f"ppo_model_exists={os.path.exists(model_path)}")
print(f"ppo_checkpoints={len(ckpts)}")
PY

echo "FINAL_EVAL_END"

FINAL_EVAL_START
root=/content/Dings_Trader/TraderHimSelf
status=OK
log_file=06_train_ppo_cpu_live.log
timesteps_last=1003520
fps_last=398.0
fps_median_last10=398.0
explained_variance_last=0.00989
approx_kl_last=0.0044323625
clip_fraction_last=0.121
ppo_model_exists=True
ppo_checkpoints=20
FINAL_EVAL_END
